In [1]:
import os
from dotenv import load_dotenv

In [2]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage, SystemMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

In [3]:
import logging

from autogen_agentchat import EVENT_LOGGER_NAME, TRACE_LOGGER_NAME

logging.basicConfig(level=logging.WARNING)

# For trace logging.
trace_logger = logging.getLogger(TRACE_LOGGER_NAME)
trace_logger.addHandler(logging.StreamHandler())
trace_logger.setLevel(logging.DEBUG)

# For structured message logging, such as low-level messages between agents.
event_logger = logging.getLogger(EVENT_LOGGER_NAME)
event_logger.addHandler(logging.StreamHandler())
event_logger.setLevel(logging.DEBUG)


In [4]:
load_dotenv(override=True)

True

In [5]:
AZURE_INFERENCE_ENDPOINT_PHI35 = os.getenv("AZURE_INFERENCE_ENDPOINT_PHI35")
AZURE_INFERENCE_CREDENTIAL_PHI35 = os.getenv("AZURE_INFERENCE_CREDENTIAL_PHI35")
AZURE_INFERENCE_ENDPOINT_PHI35_TEMPERATURE = os.getenv("AZURE_INFERENCE_ENDPOINT_PHI35_TEMPERATURE")
AZURE_INFERENCE_CREDENTIAL_PHI35_TEMPERATURE = os.getenv("AZURE_INFERENCE_CREDENTIAL_PHI35_TEMPERATURE")

In [6]:
model_client = AzureAIChatCompletionClient(
    # model="Phi-4",
    endpoint=AZURE_INFERENCE_ENDPOINT_PHI35,
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(AZURE_INFERENCE_CREDENTIAL_PHI35),
    model_info={
        "json_output": False,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
    },
)

### **Environmental Recognition Agent**
---

In [7]:
# Data to be sent to the model

# Vehicle data
VehicleData = {
    "InteriorTemperature": 25.5,  # Interior temperature (°C)
    "ExteriorTemperature": 30.2,  # Exterior temperature (°C)
    "Humidity": 60,  # Humidity (%)
    "Weather": "Sunny",  # Weather condition
    "Speed": "80km/h",  # Vehicle speed
    "Acceleration": "1.2m/s²",  # Acceleration
    "SteeringAngle": "15°",  # Steering wheel angle
    "FuelLevel": 45,  # Fuel level (%)
    "EngineRPM": 2500,  # Engine revolutions per minute (RPM)
    "BatteryVoltage": 12.8  # Battery voltage (V)
}

# Driver state
DriverState = {
    "HeartRate": 72,  # Driver's heart rate (BPM)
    "EyeStatus": "Open",  # Eye status
    "Yawning": False,  # Yawning detection
    "AttentionLevel": "High"  # Attention level
}

# Window status
WindowStatus = {
    "FrontLeft": "Closed",  # Front left window
    "FrontRight": "Closed",  # Front right window
    "RearLeft": "Open",  # Rear left window
    "RearRight": "Closed"  # Rear right window
}

# Door status
DoorStatus = {
    "FrontLeft": "Closed",  # Front left door
    "FrontRight": "Closed",  # Front right door
    "RearLeft": "Closed",  # Rear left door
    "RearRight": "Closed",  # Rear right door
    "Trunk": "Closed"  # Trunk
}

# CAN data
CANData = {
    "BrakePressure": 2.3,  # Brake pressure (bar)
    "ThrottlePosition": 25.4,  # Throttle position (%)
    "GearPosition": "Drive",  # Gear position
    "TurnSignal": "Off"  # Turn signal status
}

In [8]:
# Prompt definition
env_agent_system_message = f"""
You are an Environmental Recognition Agent.
Based on the data collected from the car's sensors, generate JSON-format data that represents the current state of the vehicle and driver.
"""

env_agent_user_message = f"""
# Input
"VehicleData": {VehicleData},
"DriverState": {DriverState},
"WindowStatus": {WindowStatus},
"DoorStatus": {DoorStatus},
"CANData": {CANData}
"""

In [9]:
env_agent = AssistantAgent(
    "env_agent",
    model_client=model_client,
    description="You are an Environmental Recognition Agent.",
    system_message=env_agent_system_message,
)

In [10]:
env_agent_response = await env_agent.on_messages(messages=[UserMessage(content=env_agent_user_message, source="user")], cancellation_token=CancellationToken())
print(env_agent_response.chat_message.content)

 {
  "VehicleData": {
    "InteriorTemperature": 25.5,
    "ExteriorTemperature": 30.2,
    "Humidity": 60,
    "Weather": "Sunny",
    "Speed": "80km/h",
    "Acceleration": "1.2m/s²",
    "SteeringAngle": "15°",
    "FuelLevel": 45,
    "EngineRPM": 2500,
    "BatteryVoltage": 12.8
  },
  "DriverState": {
    "HeartRate": 72,
    "EyeStatus": "Open",
    "Yawning": false,
    "AttentionLevel": "High"
  },
  "WindowStatus": {
    "FrontLeft": "Closed",
    "FrontRight": "Closed",
    "RearLeft": "Open",
    "RearRight": "Closed"
  },
  "DoorStatus": {
    "FrontLeft": "Closed",
    "FrontRight": "Closed",
    "RearLeft": "Closed",
    "RearRight": "Closed",
    "Trunk": "Closed"
  },
  "CANData": {
    "BrakePressure": 2.3,
    "ThrottlePosition": 25.4,
    "GearPosition": "Drive",
    "TurnSignal": "Off"
  }
}



### **Processing Agent**
---

### Temperature control agent

In [11]:
temperature_control_list_available_features = [
    {
        "type": "function",
        "function": {
            "name": "set_cabin_temperature",
            "description": "Set cabin temperature to specified degree.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "temperature": {
                        "type": "number",
                        "description": "Target temperature to be set in the cabin."
                    }
                },
                "required": ["temperature"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "auto_adjust_cabin_environment",
            "description": "Automatically adjust the cabin environment.",
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_fan_speed",
            "description": "Set fan speed to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "fan_speed": {
                        "type": "integer",
                        "enum": [1, 2, 3, 4, 5],
                        "description": "Fan speed level (1: Low, 5: High)."
                    }
                },
                "required": ["fan_speed"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_seat_heater_level",
            "description": "Set seat heater to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "level": {
                        "type": "integer",
                        "enum": [0, 1, 2, 3],
                        "description": "Seat heater level (0: Off, 3: Max)."
                    }
                },
                "required": ["level"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_seat_ventilation_level",
            "description": "Set seat ventilation to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "level": {
                        "type": "integer",
                        "enum": [0, 1, 2, 3],
                        "description": "Seat ventilation level (0: Off, 3: Max)."
                    }
                },
                "required": ["level"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_steering_heater_level",
            "description": "Set steering heater to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "level": {
                        "type": "integer",
                        "enum": [0, 1, 2],
                        "description": "Steering heater level (0: Off, 2: Max)."
                    }
                },
                "required": ["level"],
                "additionalProperties": False
            }
        }
    }
]

In [12]:
# Prompt definition
temperature_control_agent_system_message = f"""
You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\n<tools>\n{temperature_control_list_available_features}\n</tools>\nFor each function call return a json object with function name and arguments within <tool_call> </tool_call> tags with the following schema:\n<tool_call>\n{{'arguments': <args-dict>, 'name': <function-name>}}\n</tool_call>\n
"""

In [13]:
temperature_control_agent = AssistantAgent(
    "temperature_control_agent",
    model_client=model_client,
    description="You are a temperature control Agent.",
    system_message=temperature_control_agent_system_message,
)

### Entertainment control agent

In [14]:
entertainment_list_available_features = [
    {
        "type": "function",
        "function": {
            "name": "play_media",
            "description": "Plays the specified media.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "media_type": {
                        "type": "string",
                        "enum": ["music", "video", "podcast"],
                        "description": "Type of media to be played."
                    },
                    "source": {
                        "type": "string",
                        "description": "Source of the media (e.g., USB, Bluetooth, SD card)."
                    }
                },
                "required": ["media_type", "source"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "adjust_volume",
            "description": "Sets the volume level of the in-car system.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "volume": {
                        "type": "integer",
                        "minimum": 0,
                        "maximum": 100,
                        "description": "Volume level to set (0-100)."
                    }
                },
                "required": ["volume"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "select_radio_station",
            "description": "Selects a radio station by specifying its frequency.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "frequency": {
                        "type": "number",
                        "description": "Frequency of the radio station to select (e.g., 91.2)."
                    }
                },
                "required": ["frequency"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "connect_bluetooth_device",
            "description": "Connects to a Bluetooth enabled device.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "device_name": {
                        "type": "string",
                        "description": "Name of the Bluetooth device to connect."
                    }
                },
                "required": ["device_name"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "activate_voice_command",
            "description": "Activates the voice command feature to start voice operations.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "command": {
                        "type": "string",
                        "description": "Content of the voice command to be executed."
                    }
                },
                "required": ["command"],
                "additionalProperties": False
            }
        }
    }
]


In [15]:
# Prompt definition
entertainment_agent_system_message = f"""
You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\n<tools>\n{entertainment_list_available_features}\n</tools>\nFor each function call return a json object with function name and arguments within <tool_call> </tool_call> tags with the following schema:\n<tool_call>\n{{'arguments': <args-dict>, 'name': <function-name>}}\n</tool_call>\n
"""

In [16]:
entertainment_control_agent = AssistantAgent(
    "entertainment_control_agent",
    model_client=model_client,
    description="You are an entertainment control Agent.",
    system_message=entertainment_agent_system_message,
)

## Selector Group Chat

In [17]:
max_messages_termination = MaxMessageTermination(max_messages=2)

In [18]:
team = SelectorGroupChat(
    [temperature_control_agent, entertainment_control_agent],
    model_client=model_client,
    termination_condition=max_messages_termination,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [19]:
user_preferences = {
	"preferred_temperature": 20,
	"preferred_humidity": 40,
	"preferred_noise_level": "low",
	"preferred_air_quality": "good",
	"preferred_lighting": "dim",
	"preferred_music": "cheerful",
	"favorite_places": ["beach"],
}

In [20]:
user_message = f"""
# Input

## Current Environment State
{env_agent_response.chat_message.content}

## User Preferences
{user_preferences}

"""

In [21]:
await Console(team.run_stream(task=user_message))

Model selected a valid name: temperature_control_agent (attempt 1)
DEBUG:autogen_agentchat:Model selected a valid name: temperature_control_agent (attempt 1)
Selected speaker: temperature_control_agent
DEBUG:autogen_agentchat:Selected speaker: temperature_control_agent


---------- user ----------

# Input

## Current Environment State
 {
  "VehicleData": {
    "InteriorTemperature": 25.5,
    "ExteriorTemperature": 30.2,
    "Humidity": 60,
    "Weather": "Sunny",
    "Speed": "80km/h",
    "Acceleration": "1.2m/s²",
    "SteeringAngle": "15°",
    "FuelLevel": 45,
    "EngineRPM": 2500,
    "BatteryVoltage": 12.8
  },
  "DriverState": {
    "HeartRate": 72,
    "EyeStatus": "Open",
    "Yawning": false,
    "AttentionLevel": "High"
  },
  "WindowStatus": {
    "FrontLeft": "Closed",
    "FrontRight": "Closed",
    "RearLeft": "Open",
    "RearRight": "Closed"
  },
  "DoorStatus": {
    "FrontLeft": "Closed",
    "FrontRight": "Closed",
    "RearLeft": "Closed",
    "RearRight": "Closed",
    "Trunk": "Closed"
  },
  "CANData": {
    "BrakePressure": 2.3,
    "ThrottlePosition": 25.4,
    "GearPosition": "Drive",
    "TurnSignal": "Off"
  }
}


## User Preferences
{'preferred_temperature': 20, 'preferred_humidity': 40, 'preferred_noise_level': 'low', 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n# Input\n\n## Current Environment State\n {\n  "VehicleData": {\n    "InteriorTemperature": 25.5,\n    "ExteriorTemperature": 30.2,\n    "Humidity": 60,\n    "Weather": "Sunny",\n    "Speed": "80km/h",\n    "Acceleration": "1.2m/s²",\n    "SteeringAngle": "15°",\n    "FuelLevel": 45,\n    "EngineRPM": 2500,\n    "BatteryVoltage": 12.8\n  },\n  "DriverState": {\n    "HeartRate": 72,\n    "EyeStatus": "Open",\n    "Yawning": false,\n    "AttentionLevel": "High"\n  },\n  "WindowStatus": {\n    "FrontLeft": "Closed",\n    "FrontRight": "Closed",\n    "RearLeft": "Open",\n    "RearRight": "Closed"\n  },\n  "DoorStatus": {\n    "FrontLeft": "Closed",\n    "FrontRight": "Closed",\n    "RearLeft": "Closed",\n    "RearRight": "Closed",\n    "Trunk": "Closed"\n  },\n  "CANData": {\n    "BrakePressure": 2.3,\n    "ThrottlePosition": 25.4,\n    "GearPosition": "Drive",\n    "TurnSignal": "Off"\n  }\n}\n\n\n#